# Hypothesis testing
**Name(s)**: Jiaying Chen, Minh Hoang

**Website Link**: (your website link)


**Note**: Run this command to install wordcloud:
`!pip install wordcloud`

In [ ]:
!pip install wordcloud

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from wordcloud import WordCloud


import plotly.express as px
pd.options.plotting.backend = 'plotly'

# from dsc80_utils import * # Feel free to uncomment and use this.

ModuleNotFoundError: No module named 'pandas'

# Step 1: Introduction

### Loading data

In [ ]:
# Drop first 5 rows of metadata
raw = pd.read_csv('outage.csv', header=None).iloc[5:]

# Extract and clean column names from the first actual row (index 5 in the original file)
cols = raw.iloc[0, 1:].tolist()
# Drop the first column (contains "variables") and the header row itself
raw = raw.iloc[1:, 1:].copy()

# Assign cleaned column names
raw.columns = cols

# Reset index so we are working with the correct row numbers.
raw.reset_index(drop=True, inplace=True)

# Finally, drop variable column 
raw = raw.iloc[1:, :]
raw.head(5)

,OBS,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,...,POPPCT_URBAN,POPPCT_UC,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
3,3,2010,10,Minnesota,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
4,4,2012,6,Minnesota,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
5,5,2015,7,Minnesota,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


### Renaming columns

In [ ]:
new_cols = [
    'obs', 'year', 'month', 'state', 'postal_code', 'nerc_region',
    'climate_region', 'anomaly_level', 'climate_cat',
    'start_date', 'start_time', 'restore_date',
    'restore_time', 'cause_cat', 'cause_detail',
    'hurricane_names', 'duration', 'demand_loss_mw',
    'customers_affected', 'res_price', 'com_price', 'ind_price',
    'total_price', 'res_sales', 'com_sales', 'ind_sales', 'total_sales',
    'res_pct', 'com_pct', 'ind_pct', 'res_customers',
    'com_customers', 'ind_customers', 'total_customers', 'res_cust_pct',
    'com_cust_pct', 'ind_cust_pct', 'pc_realgsp_state', 'pc_realgsp_usa',
    'pc_realgsp_rel', 'pc_realgsp_change', 'util_realgsp', 'total_realgsp',
    'util_contri', 'pi_util_of_usa', 'population', 'pop_pct_urban',
    'pop_pct_uc', 'popden_urban', 'popden_uc', 'popden_rural',
    'area_pct_urban', 'area_pct_uc', 'pct_land', 'pct_water_tot',
    'pct_water_inland'
]

raw.columns = new_cols
raw.head(2)


,obs,year,month,state,postal_code,nerc_region,climate_region,anomaly_level,climate_cat,start_date,...,pop_pct_urban,pop_pct_uc,popden_urban,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


In [ ]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1534 entries, 1 to 1534
Data columns (total 56 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   obs                 1534 non-null   object
 1   year                1534 non-null   object
 2   month               1525 non-null   object
 3   state               1534 non-null   object
 4   postal_code         1534 non-null   object
 5   nerc_region         1534 non-null   object
 6   climate_region      1528 non-null   object
 7   anomaly_level       1525 non-null   object
 8   climate_cat         1525 non-null   object
 9   start_date          1525 non-null   object
 10  start_time          1525 non-null   object
 11  restore_date        1476 non-null   object
 12  restore_time        1476 non-null   object
 13  cause_cat           1534 non-null   object
 14  cause_detail        1063 non-null   object
 15  hurricane_names     72 non-null     object
 16  duration            1476

## Introduction and Question Identification
(⚠️ means still need more data, ✅ means workable with existing data )

**Currently, we are considering three problem statement to explore:**

1. ⚠️ Assessment of Infrastructure Resilience (Based on whether or not `cause_detail` has enough information.)
Analyze how different regions' infrastructure characteristics (e.g., overhead vs. underground lines, maintenance investments) correlate with outage frequency and duration. This can inform infrastructure improvement strategies.
2. ✅ Temporal Trends and Climate Change Correlation (doable with existing data, but kinda boring) 
Examine how the frequency and causes of outages have evolved over time and assess potential correlations with climate change indicators. This can provide insights into how changing climate patterns impact power reliability.
3. ⚠️ Policy and Emergency Response Evaluation (We don’t have explicit timestamps or markers indicating policy changes or emergency response dates.)
Evaluate the effectiveness of policies and emergency responses by analyzing outage data before and after the implementation of specific measures. This can guide future policy development and emergency planning.

**Ideas that are just "ok":**

⭕️ 1. Predictive Modeling of Outage Risks <u> ***(but it’s kinda boring, i bet people have done it already; not creative)*** </u>

Utilize machine learning techniques to predict the likelihood of major power outages based on factors such as weather patterns, infrastructure characteristics, and economic indicators. This can aid in proactive maintenance and resource allocation.

⭕️ 2. Socioeconomic Impact Analysis <u> ***(has been done numerous times already !!!!!!! Check out examples, two of them already exist. I know you like it, Im sry!)*** </u>

Investigate the relationship between socioeconomic factors (e.g., income levels, urbanization) and the frequency or duration of power outages. This can highlight areas where outages disproportionately affect vulnerable populations.



### Exploring idea 1
Check out whether `cause_detail` gives us any good information about the infrustructure. 

In [ ]:
raw['cause_detail'].unique()

array([nan, 'vandalism', 'heavy wind', 'thunderstorm', 'winter storm',
       'tornadoes', 'sabotage', 'hailstorm', 'uncontrolled loss',
       'winter', 'wind storm', 'computer hardware', 'public appeal',
       'storm', ' Coal', ' Natural Gas', 'hurricanes', 'wind/rain',
       'snow/ice storm', 'snow/ice ', 'transmission interruption',
       'flooding', 'transformer outage', 'generator trip',
       'relaying malfunction', 'transmission trip', 'lightning',
       'switching', 'shed load', 'line fault', 'breaker trip', 'wildfire',
       ' Hydro', 'majorsystem interruption', 'voltage reduction',
       'transmission', 'Coal', 'substation', 'heatwave',
       'distribution interruption', 'wind', 'suspicious activity',
       'feeder shutdown', '100 MW loadshed', 'plant trip', 'fog', 'Hydro',
       'earthquake', 'HVSubstation interruption', 'cables', 'Petroleum',
       'thunderstorm; islanding', 'failure'], dtype=object)

- `cause_detail` does contain some infrastructure-related failure types, like:
**transformer outage, generator trip, relaying malfunction, breaker trip, line fault, substation, transmission interruption, distribution interruption, cables, HVSubstation interruption, plant trip**, etc.
- They will allow us to indirectly infer infrastructure issues, but there are no explicit infrastructure metadata: We don’t have direct info on overhead vs underground lines, age of equipment, maintenance budgets, or investments.

- External datasets to consider: 
    - **(I FW THIS ONE HEAVY) [EIA Reports on Utility Investments](https://www.eia.gov/todayinenergy/detail.php?id=48136)** 
        - Provides financial and operational data related to maintenance and upgrades.
        - Can help explain or correlate investment levels with outage frequency/duration.
        - Good source for explaining patterns seen in outage data.
    - [Mapping the Depths: Underground Power Distribution (arXiv study, paper only)](https://arxiv.org/abs/2402.06668)
        - Unique dataset that quantifies underground vs overhead lines by utility.
        - Can provide a strong predictor variable about infrastructure type (underground = more resilient).

### Exploring EIA reports on utility investment 
-> <u> **[Annual Electric Power Industry Report, Form EIA-861 detailed data files](https://www.eia.gov/electricity/data/eia861/)**</u> <br>
-> <u> **[A Guide to EIA Electric Power Data](https://www.eia.gov/electricity/data/guide/pdf/guide.pdf)** page 9/18  </u> 
> **Retail Sales by Electric Utilities and Power Marketers (Form EIA-861, Annual Electric Power Industry Report)**
>
> Data Collected by Form EIA-861  
> The Form EIA-861, Annual Electric Power Industry Report collects annual data from a census of all utilities that sell electricity to end-use customers in the 50 states, the District of Columbia, Puerto Rico, American Samoa, the American Virgin Islands, Guam, and the Northern Mariana Islands. These surveys collect information on sales to ultimate customers by utilities and power marketers, energy efficiency programs, distributed generating capacity, and related data elements.  
>
> The data collected include several items:  
> - **Service territory by state and county**  
> - **Sales revenue to ultimate customers**  
> - **Revenue and customer count**  
> - **Source and disposition of electricity**  
> - **Advanced metering**  
> - **Demand response and energy efficiency programs**  
> - **Dynamic pricing**  
> - **Capacity and other information related to net metering**  
> - **Non-net metered distributed generating units**  
> - **Distribution system characteristics and reliability**


Out of the above, the good columns to look for are ` Distribution System Characteristics and Reliability`, ` Service Territory by State and County`, `Advanced Metering Infrastructure (AMI)`, `Revenue and Customer Count by Utility` 
-  `Distribution System Characteristics and Reliability`: Directly speaks to infrastructure resilience. 
-  `Service Territory by State and County`: Needed for merging 
- `Advanced Metering Infrastructure (AMI)`: AMI often correlates with modernization efforts and may reflect better outage response times. Can assess: Compare outage duration/frequency in regions with vs. without AMI.
- `Demand Response and Energy Efficiency Programs`: May suggest proactive infrastructure investment or mitigation strategies. Can assess: "Do regions with stronger demand response programs show fewer or shorter outages?"
- `Revenue and Customer Count by Utility`: Can assess possible correlations between revenue and investment in resilience.

### Final decision on problem statement 

<u> **Assessment of Infrastructure Resilience.   
Analyze how different regions' infrastructure maintenance investments correlate with outage frequency and duration. This can inform infrastructure improvement strategies.** </u>



# Step 2: Data Cleaning and Exploratory Data Analysis

In [ ]:
raw.head(2)

,obs,year,month,state,postal_code,nerc_region,climate_region,anomaly_level,climate_cat,start_date,...,pop_pct_urban,pop_pct_uc,popden_urban,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland
1,1,2011,7,Minnesota,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407
2,2,2014,5,Minnesota,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",...,73.27,15.28,2279,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407


## Univariate Analysis



### Countplot of climate categories

In [ ]:
climate_region_count = raw['climate_region'].value_counts().reset_index()
climate_region_count.columns = ['count', 'climate_region']
fig = px.bar(climate_region_count, x='climate_region', y='count',
            title='Outage Cause Category Count',
            labels={'cause_cat': 'Cause Category', 'count': 'Number of Outages'},
            color='climate_region',  
            color_discrete_sequence=px.colors.qualitative.Pastel  
             )

fig.show()


Log-scaled duration distribution

In [ ]:
# Changed this simple histogram for better colorings 
# tick_vals = np.arange(0, np.ceil(log_duration.max()) + 1)
# tick_text = [f"{int(np.expm1(x))}" for x in tick_vals]

# # Wrap the Series in a new dataframe
# fig = px.histogram(pd.DataFrame(log_duration), x='Log Duration of Outages', nbins=50, histnorm='percent',
#                    title='Log-Transformed Distribution of Outage Duration',
#                    color_discrete_sequence=["#BA9EF3"],  # nicer blue, you can swap colors
#                    )

# fig.update_layout(
#     xaxis=dict(title='Duration (minutes, approx)', tickvals=tick_vals, ticktext=tick_text),
#     yaxis=dict(title='% of Outages')
# )
# fig.update_traces(marker_line_width=1, marker_line_color="#FFFFFF")

# fig.show()

# Make log duration df for visualization
log_duration = np.log1p(pd.to_numeric(raw['duration'], errors='coerce'))
df = pd.DataFrame({'log_duration': log_duration})

# Create bins manually
bin_edges = np.linspace(log_duration.min(), log_duration.max(), 30)
df['bin'] = pd.cut(df['log_duration'], bins=bin_edges, include_lowest=True)
# Calculate counts per bin
bin_counts = df['bin'].value_counts().sort_index()
bin_centers = [interval.mid for interval in bin_counts.index]

hist_df = pd.DataFrame({
    'bin_center': bin_centers,
    'count': bin_counts.values
})
total_outages = df['log_duration'].notna().sum()
hist_df['percent'] = (hist_df['count'] / total_outages) * 100

tick_vals = hist_df['bin_center']
tick_text = [f"{int(np.expm1(x))}" for x in tick_vals]

fig = px.bar(
    hist_df,
    x='bin_center',
    y='percent',
    color='bin_center',  # color by bin center to get a gradient
    color_continuous_scale=px.colors.sequential.Blackbody,
    labels={'bin_center': 'Log Duration of Outages', 'percent': 'Percent of Outages'},
    title='Log Duration of Outages(Percentage)'
)

fig.update_layout(
    plot_bgcolor="#d0d5e6",

    xaxis=dict(
        title='Duration (minutes, approx)',
        tickvals=tick_vals,
        ticktext=tick_text
    )
)
fig.update_traces(
    hovertemplate=
        'Duration (log scale): %{x:.2f}<br>' +
        'Approx. Duration: %{customdata} minutes<br>' +
        'Percent of Outages: %{y:.2f}%<extra></extra>',
    customdata=[f"{int(np.expm1(x))}" for x in tick_vals]  # pass approximate durations as customdata
)

fig.show()


#### Cause detail wordcloud 

In [ ]:
raw.cause_detail
text = ' '.join(
    raw['cause_detail']
    .dropna()
    .astype(str)
    .str.lower()
    .str.strip()
    .str.replace(r'[^\w\s]', '', regex=True)  # Remove punctuation
    .str.replace(r'\s+', ' ', regex=True)
)
wordcloud = WordCloud(
    # colormap='gnuplot2',
    font_path='Impact',
    colormap='cool',
    width=800,
    height=400,
    background_color='white',
    collocations=False,  # Prevent phrases from being split
    random_state=42 
).generate(text)
wc_array = np.array(wordcloud.to_image())

fig = px.imshow(wc_array)
fig.update_layout(
    title="Word Cloud of Causes of Outages",
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False),
    margin=dict(l=10, r=10, t=40, b=10)
)
fig.show()

In [ ]:
text

'vandalism heavy wind thunderstorm winter storm tornadoes thunderstorm sabotage hailstorm hailstorm tornadoes vandalism vandalism thunderstorm uncontrolled loss vandalism uncontrolled loss winter wind storm vandalism winter computer hardware thunderstorm vandalism vandalism winter thunderstorm public appeal winter vandalism winter storm storm wind storm winter vandalism winter vandalism winter storm coal vandalism thunderstorm coal coal thunderstorm vandalism sabotage storm heavy wind vandalism vandalism coal thunderstorm natural gas vandalism thunderstorm vandalism vandalism thunderstorm hurricanes thunderstorm thunderstorm heavy wind storm winter storm winter storm winter storm winter storm storm storm thunderstorm windrain thunderstorm snowice storm thunderstorm thunderstorm winter storm vandalism snowice thunderstorm windrain thunderstorm heavy wind thunderstorm heavy wind transmission interruption thunderstorm heavy wind winter storm heavy wind vandalism winter storm flooding wind

## Bivariate Analysis

In [ ]:
fig = px.box(
    raw,
    x='state',
    y='anomaly_level',
    color='state',  # Optional: colors each box by state
    title='Anomaly Levels by State',
)

fig.update_layout(
    xaxis_title='State',
    yaxis_title='Anomaly Level',
    title_font=dict(size=30),
    xaxis_tickangle=90,
    font=dict(size=12),
    height=400,
    width=800,
    showlegend=False  # You can turn this on if coloring by state is meaningful
)

fig.show()


In [ ]:
# Create a combined label for clarity
data = raw.copy()
data['state_region'] = data['state'] + ' (' + data['nerc_region'] + ')'

fig = px.scatter(
    data,
    x='state_region',
    y='duration',
    color='year',
    title='Outage Duration by State and NERC Region (Colored by Year)',
    labels={'State_Region': 'State (NERC Region)', 'Outage Duration': 'Outage Duration (hours)'},
    hover_data=['state', 'nerc_region', 'year', 'duration'],
)

fig.update_layout(
    xaxis_tickangle=60,
    xaxis_title='State and NERC Region',
    yaxis_title='Outage Duration',
    height=700,
    width=1400,
    font=dict(size=14)
)

fig.show()


In [ ]:
# Assuming 'year' is an integer
# Convert 'year' to numeric (will turn bad values into NaN)
data['year'] = pd.to_numeric(data['year'], errors='coerce')
data['year_group'] = (data['year'] - data['year'] % 3).astype(str) + '–' + (data['year'] - data['year'] % 3 + 2).astype(str)
fig = px.box(
    data,
    x='state',
    y='duration',
    color='nerc_region',
    facet_col='year_group',
    title='Outage Duration by State (Grouped in 3-Year Periods)',
    labels={'duration': 'Outage Duration (hours)', 'year_group': 'Year Group'}
)

fig.update_layout(
    height=500,
    width=3000,
    font=dict(size=14),
    xaxis_tickangle=60
)

fig.show()



## Data cleaning

### Dropping unused features

In [ ]:
dropped = [
    'obs', 'start_date', 'start_time', 'restore_date', 'restore_time',
    'res_price', 'com_price', 'ind_price',
    'res_sales', 'com_sales', 'ind_sales',
    'res_pct', 'com_pct', 'ind_pct',
    'res_customers', 'com_customers', 'ind_customers',
    'res_cust_pct', 'com_cust_pct', 'ind_cust_pct',
    'pct_land', 'pct_water_tot', 'pct_water_inland',
    'hurricane_names', 'postal_code', 'pc_realgsp_usa', 
    'pc_realgsp_rel', 'pc_realgsp_change', 'util_realgsp',
    'total_realgsp', 'util_contri', 'nerc_region',
    'demand_loss_mw', 'customers_affected',
    'total_price', 'total_sales', 'total_customers',   
]

In [ ]:
raw_drop = raw.drop(columns = dropped)
raw_drop = raw_drop.fillna(np.nan)
raw_drop.columns

Index(['year', 'month', 'state', 'climate_region', 'anomaly_level',
       'climate_cat', 'cause_cat', 'cause_detail', 'duration',
       'pc_realgsp_state', 'pi_util_of_usa', 'population', 'pop_pct_urban',
       'pop_pct_uc', 'popden_urban', 'popden_uc', 'popden_rural',
       'area_pct_urban', 'area_pct_uc'],
      dtype='object')


# Process external data source ✅
1. Using the link on this website, download all zip files from 2000 - 2016 (Do NOT use the reformatted files!)
https://www.eia.gov/electricity/data/eia861/

2. Get file 2. Convert the file into CSV, Get the three total columns, keep columns of : Year, State, Thousands dollars, megawatthours, count for Total. Group each year's dataframe by state, sum for each column. 

3. For now, just have 17 dataframes.

Total revenue is very high, might want to regularize it?

In [ ]:
external = pd.DataFrame(columns = ['state', 'total_revenue', 'megawatthours', 'count', 'year'])

for i in range(17):
    if i < 10:
        num = '0' + str(i)
    else:
        num = str(i)
    state_col = 'Unnamed: 3' if i == 0 else 'Unnamed: 6'
    df = pd.read_csv(f'20{num}.csv')
    counts = df.columns[-1]
    megawatts = df.columns[-2]
    revenue = df.columns[-3]
    df = df.iloc[2:][[state_col, revenue, megawatts, counts]].rename(columns = {state_col: 'state', revenue: 'total_revenue', 
                                                                                            megawatts: 'megawatthours', counts: 'count'})
    df['total_revenue'] = df['total_revenue'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)*1000
    df['megawatthours'] = df['megawatthours'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)
    df['count'] = df['count'].str.replace(',', '', regex = True).replace(['.', ''], np.nan).astype(float)
    df = df.groupby('state').sum().reset_index()
    df['year'] = '20' + num
    external = pd.concat([external, df], axis = 0)

external['year'] = external['year'].astype(int) 

C:\Users\monso\AppData\Local\Temp\ipykernel_17876\1914324587.py:20: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [ ]:
external['year_state'] = external['year'].astype(str) + external['state'].astype(str)
external.head()

In [ ]:
state_duration = raw.groupby(['year', 'postal_code'])['duration'].sum().reset_index().rename(columns = {'postal_code': 'state'})
state_duration['year_state'] = state_duration['year'].astype(str) + state_duration['state'].astype(str)
final_df = external.merge(state_duration, on = 'year_state', how = 'left')[['total_revenue', 'megawatthours', 'count', 'year_state', 'duration']].fillna(0)
final_df['year'] = final_df['year_state'].str[:4]
final_df['state'] = final_df['year_state'].str[4:]
final_df = final_df.drop(columns = 'year_state')
final_df.to_csv('raw_external_combined.csv')

# [TODO] Inspect and clean missing data from original dataframe 
1. Inspect how much missing data there is in each column (percentage) (rename the dataframe to outage, so that there's destintion between raw dataset (used for missingness later) and real dataset we will be using)
2. Decide on how to fill in missingness (drop? mean imputation? probability imputation?)
3. Feature engineer however you think is fit. 

In [ ]:
(raw_drop.isna().sum(axis = 0) / raw_drop.shape[0]).sort_values(ascending= False)

cause_detail        0.307040
duration            0.037810
popden_uc           0.006519
popden_rural        0.006519
anomaly_level       0.005867
month               0.005867
climate_cat         0.005867
climate_region      0.003911
year                0.000000
state               0.000000
cause_cat           0.000000
pi_util_of_usa      0.000000
pc_realgsp_state    0.000000
population          0.000000
pop_pct_urban       0.000000
popden_urban        0.000000
pop_pct_uc          0.000000
area_pct_urban      0.000000
area_pct_uc         0.000000
dtype: float64

### Columns with notably large amounts of missing data:  
demand_loss_mw  
customers_affected  
cause_detail  
hurricane_names  


# Step 4: Hypothesis testing
Recall our problem statement:   
**Assessment of Infrastructure Resilience (Based on whether or not cause_detail has enough information.) Analyze how different regions' infrastructure <u>*maintenance investments*</u> correlate with outage frequency and duration. This can inform infrastructure improvement strategies, based on yearly spent on utility for each state**

### Requirements: 
1. Clearly state a pair of hypotheses and perform a hypothesis test or permutation test that is not related to missingness. 
2. Clearly state your null and alternative hypotheses, your choice of test statistic and significance level, the resulting 
p -value, and your conclusion. Justify why these choices are good choices for answering the question you are trying to answer.

Optional: Embed a visualization related to your hypothesis test in your website.

Tip: When making writing your conclusions to the statistical tests in this project, never use language that implies an absolute conclusion; since we are performing statistical tests and not randomized controlled trials, we cannot prove that either hypothesis is 100% true or false.

In [ ]:
util_invest = pd.read_csv('external.csv')
util_invest.drop(columns=['Unnamed: 0'], inplace=True)
util_invest.rename(columns={'count':'customer_count'}, inplace=True)
util_invest['year'] = util_invest['year'].astype(str)

In [ ]:
util_invest.head()

,state,total_revenue,megawatthours,customer_count,year
0,AK,5.352460e+08,5309970.0,273530.0,2000
1,AL,4.687257e+09,83524220.0,2262753.0,2000
2,AR,2.399365e+09,41611188.0,1362838.0,2000
3,AZ,4.431208e+09,61130045.0,2193562.0,2000
4,CA,2.289222e+10,264464209.0,12742304.0,2000


In [ ]:
# merge dataframes 

raw = raw.drop(columns='state').rename(columns={'postal_code': 'state'})
raw_w_util = raw.merge(util_invest, how='left', on=['year', 'state'])
raw_w_util.head()

,obs,year,month,state,nerc_region,climate_region,anomaly_level,climate_cat,start_date,start_time,...,popden_uc,popden_rural,area_pct_urban,area_pct_uc,pct_land,pct_water_tot,pct_water_inland,total_revenue,megawatthours,customer_count
0,1,2011,7,MN,MRO,East North Central,-0.3,normal,"Friday, July 1, 2011",5:00:00 PM,...,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407,5.928515e+09,68532708.0,2595696.0
1,2,2014,5,MN,MRO,East North Central,-0.1,normal,"Sunday, May 11, 2014",6:38:00 PM,...,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407,6.540932e+09,68719367.0,2640737.0
2,3,2010,10,MN,MRO,East North Central,-1.5,cold,"Tuesday, October 26, 2010",8:00:00 PM,...,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407,5.701194e+09,67799706.0,2586905.0
3,4,2012,6,MN,MRO,East North Central,-0.1,normal,"Tuesday, June 19, 2012",4:30:00 AM,...,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407,5.976656e+09,67144577.0,2606809.0
4,5,2015,7,MN,MRO,East North Central,1.2,warm,"Saturday, July 18, 2015",2:00:00 AM,...,1700.5,18.2,2.14,0.6,91.5926658691451,8.40733413085488,5.47874298334407,6.346338e+09,66579234.0,2673531.0


In [ ]:
raw_w_util[['state','year','duration', 'total_revenue', 'megawatthours','customer_count']]

,state,year,duration,total_revenue,megawatthours,customer_count
0,MN,2011,3060,5.928515e+09,68532708.0,2595696.0
1,MN,2014,1,6.540932e+09,68719367.0,2640737.0
2,MN,2010,3000,5.701194e+09,67799706.0,2586905.0
3,MN,2012,2550,5.976656e+09,67144577.0,2606809.0
4,MN,2015,1740,6.346338e+09,66579234.0,2673531.0
...,...,...,...,...,...,...
1529,ND,2011,720,1.030332e+09,13737016.0,394394.0
1530,ND,2006,NaN,6.980870e+08,11245238.0,366037.0
1531,SD,2009,59,8.131895e+08,11010118.0,436229.0
1532,SD,2009,181,8.131895e+08,11010118.0,436229.0


### 1. Creating a Single “Spending” Feature for Each State-Year
We will create a heuristic feature that combines normalized revenue, megawatthours, and customer count.
1. Normalized Revenue per Customer: how much money each customer generates — a proxy for investment per capita.
2. Normalized Energy Usage per Customer: how much energy each customer consumes — a proxy for infrastructure demand.

Scaling matters: Without normalization, one metric could dominate due to its magnitude.

Per-customer makes it equitable: Larger states won't automatically look better just because they're bigger.

Directly interpretable: States with higher infra scores are funding and supplying more power per user.



In [2]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
rw_ht = raw_w_util.copy()

rw_ht['rev_per_cust'] = rw_ht['total_revenue'] / rw_ht['customer_count']
rw_ht['mwh_per_cust'] = rw_ht['megawatthours'] / rw_ht['customer_count']

rw_ht[['rev_cust_norm', 'mwh_cust_norm']] = scaler.fit_transform(
    rw_ht[['rev_per_cust', 'mwh_per_cust']]
)

rw_ht['infra_score'] = (rw_ht['rev_cust_norm'] + rw_ht['mwh_cust_norm']) / 2
rw_ht[['infra_score', 'state', 'duration']]

ModuleNotFoundError: No module named 'sklearn'

Null (H₀): High- and low-spending states have equal or longer outage durations.   
Alternative (H₁): High-spending states have shorter outage durations.

In [3]:
threshold = rw_ht['infra_score'].median()
rw_ht['spending_group'] = rw_ht['infra_score'].apply(lambda x: 'high' if x >= threshold else 'low')

df = rw_ht.dropna(subset=['duration'])
high_mean = df[df['spending_group'] == 'high']['duration'].astype(int).mean()
low_mean = df[df['spending_group'] == 'low']['duration'].astype(int).mean()
observed_diff = low_mean - high_mean  # Expect positive if high spenders are better

n_permutations = 5000
perm_diffs = []

for _ in range(n_permutations):
    shuffled = np.random.permutation(df['spending_group'].values)  # shuffle group labels
    df_shuffled = df.copy()
    df_shuffled['shuffled_group'] = shuffled  # assign shuffled group

    high = df_shuffled[df_shuffled['shuffled_group'] == 'high']['duration'].astype(float)
    low = df_shuffled[df_shuffled['shuffled_group'] == 'low']['duration'].astype(float)

    diff = np.mean(low) - np.mean(high)
    perm_diffs.append(diff)

p_value = np.mean(np.array(perm_diffs) >= observed_diff)


NameError: name 'rw_ht' is not defined

In [4]:
p_value

NameError: name 'p_value' is not defined

In [5]:
raw.info()

NameError: name 'raw' is not defined

### 2. Choosing the Outcome Variable — Count vs Duration

### 3: Formulating Hypotheses and Test Statistics